In [302]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

<Strong> A basic machine learning approach. </Strong>

<p> I am going to look into cheap machine learning models such as this small densley connected netowrk before looking into complicated and computationally expensive TCN models. </p>

In [303]:
import pandas as pd
import numpy as np

In [304]:
df = pd.read_csv("../../../cryptoData/BTC_15m_data.csv")

In [305]:
df

,date,open,high,low,close,volume
0,2017-08-17 04:00:00,4261.48,4280.56,4261.48,4261.48,2.189061
1,2017-08-17 04:15:00,4261.48,4270.41,4261.32,4261.45,9.119865
2,2017-08-17 04:30:00,4280.00,4310.07,4267.99,4310.07,21.923552
3,2017-08-17 04:45:00,4310.07,4313.62,4291.37,4308.83,13.948531
4,2017-08-17 05:00:00,4308.83,4328.69,4304.31,4304.31,5.101153
...,...,...,...,...,...,...
155486,2022-01-28 14:45:00,36684.11,36828.38,36600.00,36759.65,560.007760
155487,2022-01-28 15:00:00,36759.65,37198.00,36682.24,37149.99,649.317780
155488,2022-01-28 15:15:00,37150.00,37300.00,36925.89,36988.91,1061.113920
155489,2022-01-28 15:30:00,36985.43,37115.32,36874.46,37022.42,452.420950


In [306]:
float_data = df.values[:, 1: ,].astype(float)
closing_data = float_data[:,3,]

<Strong> Just grabbing the Closing price , using Omnivariable Prediction.</Strong>

In [307]:
mean = closing_data[:200000].mean(axis=0)
closing_data -= mean
std = closing_data[:200000].std(axis=0)
closing_data /= std

print(closing_data.shape[-1])

155491


<Strong> Temporal so doesn't shuffle (avoids leakage). </Strong>

In [308]:
train, test = temporal_train_test_split(closing_data, test_size = 25491)

<p> Implementing my rolling window technique as suggested in towardsdatascience article. </p>

In [309]:
#Found this code on Kaggle 

In [310]:
fh = np.arange(1 , len(test) + 1)
fh

array([    1,     2,     3, ..., 25489, 25490, 25491])

In [311]:
#Takes in single series , window length , and forecasting horizon
def split_into_train_test(data , in_num , fh):
    train,test = data[:-fh] , data[-(fh+ in_num):]
    x_train,y_train = train[:-1] , np.roll(train , -in_num)[:-in_num]
    x_test , y_test = test[:-1] , np.roll(test,-in_num)[:-in_num]
    
    x_train = np.reshape(x_train , (-1 ,1))
    x_test = np.reshape(x_test , (-1 , 1))
    temp_test = np.roll(x_test , -1)
    temp_train = np.roll(x_train , -1)
    
    for x in range (1 , in_num):
        x_train = np.concatenate((x_train[:-1] , temp_train[:-1]) , 1)
        x_test = np.concatenate((x_test[:-1] , temp_test[:-1]) , 1)
        temp_test = np.roll(temp_test, -1)[:-1]
        temp_train=np.roll(temp_train , -1)[:-1]
        
    return x_train , y_train , x_test , y_test

In [312]:
x_train , y_train , x_test , y_test = split_into_train_test(closing_data , 10 , len(fh))

In [313]:
print(x_train.shape , y_train.shape)
print(x_test.shape , y_test.shape)

(129990, 10) (129990,)
(25491, 10) (25491,)


In [314]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

model.build((None , 10))

In [315]:
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_77 (Dense)            (None, 2)                 22        
                                                                 
 dense_78 (Dense)            (None, 3)                 9         
                                                                 
 dense_79 (Dense)            (None, 4)                 16        
                                                                 
Total params: 47
Trainable params: 47
Non-trainable params: 0
_________________________________________________________________


In [316]:
model.compile(optimizer=RMSprop() , loss = 'mae')

In [318]:
history = model.fit(x_train , y_train)

4063/4063 [==============================] - 3s 824us/step - loss: 0.0073 0s - loss: 0.0


In [320]:
history.history

{'loss': [0.007331273052841425]}

In [ ]:
predictions = []
last_window = x_train[-1,:].reshape(1,-1)

last_prediction = model.predict(last_window)[0]

for i in range(len(fh)):
    
    predictions.append(last_prediction)
    last_window[0] = np.roll(last_window[0], -1)    
    last_window[0 , (len(last_window[0]) -1)] = last_prediction
    last_prediction = model.predict(last_window)[0]
    
y_prec_rec = pd.Series(predictions , index = y_test.index)
smape_loss(y_pred_rec , y_test)
    

In [184]:
def rolling_window(train_data , window_size , predict_future_steps):
    
    for i in range (len(train_data) - window_size - predict_future_steps + 1):
        
        split_train = train_data[i:window_size+i]
        split_val = train_data[i+window_size:window_size+i+predict_future_steps]
        
        yield split_train , split_val

In [185]:
window_size = 10
size_of_prediction = 1
rolling = rolling_window(train , window_size , size_of_prediction)

In [186]:
for train , val in rolling:
    print(train.shape)
